1. Loading and processing data 

In [86]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score, KFold, cross_val_predict
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.cluster import KMeans
import numpy as np

In [87]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
params = {
    # melbourne coordinates
	"latitude": -37.814,
	"longitude": 144.9633,
	"start_date": "2020-01-01",
	"end_date": "2024-12-25",
	"daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "sunrise", "sunset", "uv_index_max", "precipitation_sum", "wind_speed_10m_max", "wind_gusts_10m_max"],
	"timezone": "Australia/Sydney"
}
responses = openmeteo.weather_api(url, params=params)

In [88]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates -37.75°N 144.875°E
Elevation 19.0 m asl
Timezone b'Australia/Sydney' b'AEDT'
Timezone difference to GMT+0 39600 s


In [89]:
# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
daily_sunrise = daily.Variables(3).ValuesAsNumpy()
daily_sunset = daily.Variables(4).ValuesAsNumpy()
daily_uv_index_max = daily.Variables(5).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(6).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(7).ValuesAsNumpy()
daily_wind_gusts_10m_max = daily.Variables(8).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["sunrise"] = daily_sunrise
daily_data["sunset"] = daily_sunset
daily_data["uv_index_max"] = daily_uv_index_max
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
daily_data["ave_temp"] = (daily_temperature_2m_max + daily_temperature_2m_min)/2

daily_dataframe = pd.DataFrame(data = daily_data)

2. Cleaning data

In [90]:
# input features: sunrise, sunset, uv, precipitation, wind speed and gust
features = daily_dataframe[["sunrise", "sunset", "uv_index_max", "precipitation_sum", "wind_speed_10m_max", "wind_gusts_10m_max", "ave_temp", "weather_code"]]

# cannot use rows with missing features, so remove
missing_rows = features[features.isna().any(axis=1)]

# only train model using rows that contain all features
valid_rows = features[features.notna().all(axis=1)]

3a. Predicting temperature

In [91]:
# use cross validation to train, test and evaluate the model
X = valid_rows[["uv_index_max", "precipitation_sum", "wind_speed_10m_max", "wind_gusts_10m_max"]]
Y = valid_rows["ave_temp"]
temp_models = dict()

In [92]:
# model 0: baseline model mean

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
mean_prediction = np.mean(Y_train)
baseline_predictions = np.full_like(Y_test, mean_prediction, dtype=np.float64)
mse = mean_squared_error(Y_test, baseline_predictions)
r2 = r2_score(Y_test, baseline_predictions)

temp_models["model0"] = r2

print(f"Baseline Model Mean Squared Error: {mse:.4f}")
print(r2)

Baseline Model Mean Squared Error: 21.0693
-0.0017689669822316123


In [93]:
# model 1: simple linear regression
model1 = LinearRegression()

# Set up cross-validation (e.g., 5-fold cross-validation)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
scores = cross_val_score(model1, X, Y, cv=cv, scoring='r2')
predictions = cross_val_predict(model1, X, Y, cv=cv)

temp_models[model1] = float(np.mean(scores))
print(temp_models)

# Print cross-validation results
print("Cross-validation scores: ", scores)
print("Mean r2: ", np.mean(scores))
print("Standard deviation: ", np.std(scores))

results1 = pd.DataFrame({'Actual Values': Y, 'Predicted Values': predictions})
print(results1)


{'model0': -0.0017689669822316123, LinearRegression(): 0.4567053556442261}
Cross-validation scores:  [0.42419428 0.5160076  0.4393962  0.44142914 0.46249956]
Mean r2:  0.4567053556442261
Standard deviation:  0.03206327842946231
      Actual Values  Predicted Values
448       16.845001          9.840088
449       17.945000         16.849234
450       17.645000         16.062347
451       17.795000         17.189152
452       16.744999         15.125054
...             ...               ...
1816      17.917000         20.931850
1817      16.392000         20.946257
1818      14.417000         19.867306
1819      17.017000         20.413437
1820      23.767000         20.278965

[1373 rows x 2 columns]


In [94]:
# model 2: random forest regressor
model2 = RandomForestRegressor()

# Set up cross-validation (e.g., 5-fold cross-validation)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
scores = cross_val_score(model2, X, Y, cv=cv, scoring='r2')
predictions = cross_val_predict(model2, X, Y, cv=cv)

temp_models[model2] = float(np.mean(scores))

# Print cross-validation results
print("Cross-validation scores: ", scores)
print("Mean r2: ", np.mean(scores))
print("Standard deviation: ", np.std(scores))

results2 = pd.DataFrame({'Actual Values': Y, 'Predicted Values': predictions})
print(results2)

Cross-validation scores:  [0.38448389 0.4860208  0.44548204 0.41289272 0.425315  ]
Mean r2:  0.4308388905807042
Standard deviation:  0.03394356938248215
      Actual Values  Predicted Values
448       16.845001         10.613400
449       17.945000         14.534430
450       17.645000         14.763280
451       17.795000         14.824360
452       16.744999         15.301920
...             ...               ...
1816      17.917000         21.879400
1817      16.392000         23.064911
1818      14.417000         22.083200
1819      17.017000         21.117751
1820      23.767000         22.270420

[1373 rows x 2 columns]


In [95]:
# model 3: KNN
model3 = KNeighborsRegressor()

# Set up cross-validation (e.g., 5-fold cross-validation)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
scores = cross_val_score(model3, X, Y, cv=cv, scoring='r2')
predictions = cross_val_predict(model3, X, Y, cv=cv)

temp_models[model3] = float(np.mean(scores))

# Print cross-validation results
print("Cross-validation scores: ", scores)
print("Mean r2: ", np.mean(scores))
print("Standard deviation: ", np.std(scores))

results3 = pd.DataFrame({'Actual Values': Y, 'Predicted Values': predictions})
print(results3)

Cross-validation scores:  [0.31978613 0.35005265 0.38050818 0.33395106 0.30518562]
Mean r2:  0.337896728515625
Standard deviation:  0.025988773187259527
      Actual Values  Predicted Values
448       16.845001         11.775000
449       17.945000         14.079999
450       17.645000         15.508799
451       17.795000         13.508800
452       16.744999         13.455000
...             ...               ...
1816      17.917000         18.622601
1817      16.392000         22.682001
1818      14.417000         22.257603
1819      17.017000         18.786999
1820      23.767000         21.271999

[1373 rows x 2 columns]


3b. Predicting weather codes

In [96]:
# use cross validation to train, test and evaluate the model
X = valid_rows[["uv_index_max", "precipitation_sum", "wind_speed_10m_max", "wind_gusts_10m_max"]]
Y = valid_rows["weather_code"]
weather_models = dict()

In [97]:
# model 0: Baseline Dummy Classifier

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
model0 = DummyClassifier() # uses the most frequent label
model0.fit(X_train, Y_train)
model0_predictions = model0.predict(X_test)
accuracy = accuracy_score(Y_test, model0_predictions)

weather_models["model0"] = accuracy

print(f"Baseline Model Accuracy: {accuracy:.4f}")

Baseline Model Accuracy: 0.4109


In [98]:
# model 1: Random Forest Classifier

model1 = RandomForestClassifier()

# Set up cross-validation (e.g., 5-fold cross-validation)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
scores = cross_val_score(model1, X, Y, cv=cv, scoring='accuracy')
predictions = cross_val_predict(model1, X, Y, cv=cv)

weather_models[model1] = float(np.mean(scores))

# Print cross-validation results
print("Cross-validation scores: ", scores)
print("Mean accuracy: ", np.mean(scores))
print("Standard deviation: ", np.std(scores))

results1 = pd.DataFrame({'Actual Values': Y, 'Predicted Values': predictions})
print(results1)

Cross-validation scores:  [0.57090909 0.59636364 0.56727273 0.5729927  0.60583942]
Mean accuracy:  0.5826755142667551
Standard deviation:  0.015449158401388653
      Actual Values  Predicted Values
448            55.0              80.0
449            53.0              51.0
450             3.0               3.0
451            81.0              53.0
452            51.0              51.0
...             ...               ...
1816            3.0               3.0
1817           80.0              80.0
1818           80.0              80.0
1819            3.0               3.0
1820            3.0               3.0

[1373 rows x 2 columns]


4. Determine best ML algorithm

In [105]:
def get_dict_key(dictionary, value):
    for key, val in dictionary.items():
        if value == val:
            return key

# best temp model is one with highest r2 score (regression)
best_temp_val = max(temp_models.values())
best_temp_model = get_dict_key(temp_models, best_temp_val)
print(best_temp_val)
print(best_temp_model)

# best weather code model is one with highest accuracy score (classifcation)
best_weather_val = max(weather_models.values())
best_weather_model = get_dict_key(weather_models, best_weather_val)
print(best_weather_val)
print(best_weather_model)


{'model0': -0.0017689669822316123, LinearRegression(): 0.4567053556442261, RandomForestRegressor(): 0.4308388905807042, KNeighborsRegressor(): 0.337896728515625}
0.4567053556442261
LinearRegression()
{'model0': 0.4109090909090909, RandomForestClassifier(): 0.5826755142667551}
0.5826755142667551
RandomForestClassifier()
